In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import BoundaryNorm
from scipy.interpolate import griddata  
import numpy.ma as ma
import folium
from folium.plugins import HeatMap
from mpltools import color

In [2]:
dadosFull = pd.read_csv("E:/Mestrado Dados Climatologicos/Historicos/Consolidados/2020_Full.csv",";");  

In [11]:
meteoro_group2 = dadosFull.groupby(['DATA','NAME_FILE'],as_index=False).agg({'PRECIPITACAO':'sum','UMIDADE_MIN':'mean','UMIDADE_MAX':'mean','TEMPERATURA_MAX':'mean','TEMPERATURA_MIN':'mean','VENTO_VELOCIDADE':'mean','RADIACAO_GLOBAL':'mean'});

In [14]:
meteoro_group2.head();
meteoro_group2.to_csv("E:/Mestrado Dados Climatologicos/Historicos/Consolidados/2020_Full_Consolidados.csv",index=False,sep =';');

In [10]:
dadosFull[dadosFull.columns[0]].count()

5173776

In [25]:
print(dadosFull.isnull().sum()) 


DATA                      0
PRECIPITACAO        1544399
RADIACAO_GLOBAL     2871561
TEMPERATURA_MAX     1185191
TEMPERATURA_MIN     1185266
UMIDADE_MAX         1355276
UMIDADE_MIN         1359753
VENTO_VELOCIDADE    1329355
NAME_FILE                 0
dtype: int64


,DATA,PRECIPITACAO,RADIACAO_GLOBAL,TEMPERATURA_MAX,TEMPERATURA_MIN,UMIDADE_MAX,UMIDADE_MIN,VENTO_VELOCIDADE,NAME_FILE


In [24]:
dadosFull[dadosFull['PRECIPITACAO'].isnull()].groupby(['NAME_FILE'],as_index=False).count()

,NAME_FILE,DATA,PRECIPITACAO,RADIACAO_GLOBAL,TEMPERATURA_MAX,TEMPERATURA_MIN,UMIDADE_MAX,UMIDADE_MIN,VENTO_VELOCIDADE
0,INMET_CO_DF_A042_BRAZLANDIA_01012020_A_3112202...,179,0,0,0,0,0,0,0
1,INMET_CO_DF_A045_AGUAS EMENDADAS_01012020_A_31...,176,0,1,2,2,2,2,1
2,INMET_CO_DF_A046_GAMA (PONTE ALTA)_01012020_A_...,63,0,0,0,0,0,0,0
3,INMET_CO_DF_A047_PARANOA (COOPADF)_01012020_A_...,176,0,0,0,0,0,0,0
4,INMET_CO_GO_A002_GOIANIA_01012020_A_31122020.CSV,107,0,40,71,71,71,71,71
...,...,...,...,...,...,...,...,...,...
516,INMET_S_SC_A867_ARARANGUA_01012020_A_31122020.CSV,9,0,0,0,0,0,0,0
517,INMET_S_SC_A868_ITAJAI_01012020_A_31122020.CSV,1511,0,0,0,0,0,0,0
518,INMET_S_SC_A870_RANCHO QUEIMADO_01012020_A_311...,5346,0,73,133,133,105,105,133
519,INMET_S_SC_A895_CHAPECO_01012020_A_31122020.CSV,8,0,0,0,0,0,0,0


In [21]:
print(dadosFull.groupby(['NAME_FILE'],as_index=False).agg({'PRECIPITACAO':'sum'}))

                                             NAME_FILE  PRECIPITACAO
0    INMET_CO_DF_A001_BRASILIA_01012020_A_31122020.CSV        1576.6
1    INMET_CO_DF_A042_BRAZLANDIA_01012020_A_3112202...        2091.6
2    INMET_CO_DF_A045_AGUAS EMENDADAS_01012020_A_31...        1575.6
3    INMET_CO_DF_A046_GAMA (PONTE ALTA)_01012020_A_...        1951.2
4    INMET_CO_DF_A047_PARANOA (COOPADF)_01012020_A_...        2180.8
..                                                 ...           ...
584  INMET_S_SC_A867_ARARANGUA_01012020_A_31122020.CSV        1368.0
585     INMET_S_SC_A868_ITAJAI_01012020_A_31122020.CSV         985.6
586  INMET_S_SC_A870_RANCHO QUEIMADO_01012020_A_311...         808.0
587    INMET_S_SC_A895_CHAPECO_01012020_A_31122020.CSV        1600.4
588  INMET_S_SC_A898_CAMPOS NOVOS_01012020_A_311220...        1761.4

[589 rows x 2 columns]


In [31]:
dadosFull[dadosFull['VENTO_VELOCIDADE'] < 0]

,DATA,PRECIPITACAO,RADIACAO_GLOBAL,TEMPERATURA_MAX,TEMPERATURA_MIN,UMIDADE_MAX,UMIDADE_MIN,VENTO_VELOCIDADE,NAME_FILE
